In [1]:
import configparser
from datetime import datetime
import os
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format

### Get Keys

In [2]:
config = configparser.ConfigParser()
config.read('dl.cfg')

os.environ['AWS_ACCESS_KEY_ID']=config['AWS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS']['AWS_SECRET_ACCESS_KEY']

### Create Spark Session

In [3]:
def create_spark_session():
    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.5") \
        .getOrCreate()
    return spark

In [4]:
spark = create_spark_session()

### Load Data

#### 1. Song Data

In [5]:
song_data = "s3a://udacity-dend/song-data/A/B/C/*.json"
output_data = "s3a://udacity-my-bucket/"

In [6]:
df=spark.read.json(song_data)

Py4JJavaError: An error occurred while calling o30.json.
: java.lang.NumberFormatException: For input string: "100M"
	at java.lang.NumberFormatException.forInputString(NumberFormatException.java:65)
	at java.lang.Long.parseLong(Long.java:589)
	at java.lang.Long.parseLong(Long.java:631)
	at org.apache.hadoop.conf.Configuration.getLong(Configuration.java:1538)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.initialize(S3AFileSystem.java:248)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3303)
	at org.apache.hadoop.fs.FileSystem.access$200(FileSystem.java:124)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3352)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3320)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:479)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:361)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:46)
	at org.apache.spark.sql.execution.datasources.DataSourc

In [7]:
print(df.count())
df.printSchema()

NameError: name 'df' is not defined

In [9]:
df.show(5)

+------------------+---------------+--------------------+----------------+--------------------+---------+---------+------------------+--------------------+----+
|         artist_id|artist_latitude|     artist_location|artist_longitude|         artist_name| duration|num_songs|           song_id|               title|year|
+------------------+---------------+--------------------+----------------+--------------------+---------+---------+------------------+--------------------+----+
|ARA04401187B991E6E|       54.99241|Londonderry, Nort...|        -7.31923|JOSEF LOCKE & ORC...|184.16281|        1|SOXKFTF12A6D4FBF31|Isle Of Innisfree...|   0|
|ARRZ1VV1187FB39C73|       33.52029|      Birmingham, AL|        -86.8115|DENNIS EDWARDS FE...|240.14322|        1|SOCSUKH12A67AE08C7|Don't Look Any Fu...|   0|
|ARNU0OM1187FB3F14A|       32.77815|          Dallas, TX|        -96.7954|Larry Groce/Disne...| 90.04363|        1|SOPEJZP12A8C1369E6|He's Got The Whol...|   0|
|AR98JLC1187B9ADE23|       34.0534

In [12]:
pd.set_option('max_colwidth', 200)
df.limit(5).toPandas()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARA04401187B991E6E,54.99241,"Londonderry, Northern Ireland",-7.31923,JOSEF LOCKE & ORCHESTRA,184.16281,1,SOXKFTF12A6D4FBF31,Isle Of Innisfree (Film 'Quiet Man') (1992 Digital Remaster),0
1,ARRZ1VV1187FB39C73,33.52029,"Birmingham, AL",-86.81150,DENNIS EDWARDS FEAT. THE WEATHER GIRLS,240.14322,1,SOCSUKH12A67AE08C7,Don't Look Any Further - 13 Studio Extended Version,0
2,ARNU0OM1187FB3F14A,32.77815,"Dallas, TX",-96.79540,Larry Groce/Disneyland Children's Sing-Along Chorus,90.04363,1,SOPEJZP12A8C1369E6,He's Got The Whole World In His Hands,0
3,AR98JLC1187B9ADE23,34.05349,"Los Angeles, California, United States",-118.24532,Megadeth,243.06893,1,SOUJRGA12A6310EC37,Bad Omen (24-Bit Digitally Remastered 04) (),0
4,ARFU3UI1187B9A7372,50.82821,Brighton / Los Angeles United Kingdom,-0.13449,Che Sudaka,150.25587,1,SOYWXKY12A6D4FB1A4,Somos Más - Qué Difícil,2005


In [7]:
df.createOrReplaceTempView("songs")

In [8]:
songs_table = spark.sql("""
    SELECT DISTINCT 
        song_id,
        title,
        artist_id,
        year,
        duration
    FROM songs
""")

In [24]:
pd.set_option('max_colwidth', 200)
songs_table.limit(5).toPandas()

,song_id,title,artist_id,year,duration
0,SOCEMJV12A6D4F7667,Giant Steps (Alternate Version_ Take 5_ Alternate),ARIOZCU1187FB3A3DC,0,220.44689
1,SOQFYBD12AB0182188,Intro,ARAADXM1187FB3ECDB,1999,67.63057
2,SOTDCIR12AB0184574,I Need A Mother,ARZGTK71187B9AC7F5,2010,158.01424
3,SOUPIRU12A6D4FA1E1,Der Kleine Dompfaff,ARJIE2Y1187B994AB7,0,152.92036
4,SODWBIK12AB017F87D,Aüita,ARSMG8X1187B99CA99,2009,210.10240


In [76]:
#songs_table.write.partitionBy("year", "artist_id").mode('overwrite').parquet("songs.parquet")

In [12]:
songs_data= os.path.join(output_data, "songs")
songs_table.write.partitionBy("year", "artist_id").parquet(songs_data, mode="overwrite")

In [13]:
artists_table = spark.sql("""
    SELECT DISTINCT 
        artist_id,
        artist_name as name,
        artist_location as location, 
        artist_latitude as lattitude,
        artist_longitude as longitude
    FROM songs
""")

In [95]:
#artists_table.write.mode("overwrite").parquet("artists.parquet")

In [14]:
artists_data=os.path.join(output_data, "artists")
artists_table.write.mode("overwrite").parquet(artists_data)

#### 2. Log Data

In [15]:
log_data = "s3a://udacity-dend/log_data/2018/11/2018-11-12-events.json"

In [36]:
log_data = "s3a://udacity-dend/log_data/*/*/*.json"

In [37]:
df=spark.read.json(log_data)

In [24]:
print(df.count())
df.printSchema()

213
root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [17]:
df=df.filter(df.page == 'NextSong')

In [18]:
df.createOrReplaceTempView("logs")

In [19]:
users_table = spark.sql("""
    SELECT DISTINCT 
        userId as user_id,
        firstName as first_name,
        lastName as last_name,
        gender,
        level
    FROM logs
""")

In [18]:
#users_table.write.mode("overwrite").parquet("users.parquet")

In [20]:
users_data=os.path.join(output_data, "users")
users_table.write.mode("overwrite").parquet(users_data)

In [21]:
from pyspark.sql.types import IntegerType, TimestampType
from pyspark.sql.functions import from_unixtime

get_timestamp = udf(lambda x: int(x/1000),IntegerType())
df = df.withColumn('start_time',get_timestamp('ts'))

get_datetime = udf(lambda x: from_unixtime(x),TimestampType())
df = df.withColumn('datetime',from_unixtime('start_time'))

In [22]:
time_table = df.select(col('datetime').alias('start_time'),
                       hour('datetime').alias('hour'),
                       dayofmonth('datetime').alias('day'),
                       weekofyear('datetime').alias('week'),
                       month('datetime').alias('month'),
                       year('datetime').alias('year'),
                       date_format('datetime','E').alias('weekday'))

In [22]:
#time_table.write.partitionBy('year', 'month').parquet(os.path.join(output_data, 'time'), 'overwrite')

In [23]:
#time_table.write.partitionBy('year','month').parquet("time.parquet",mode="overwrite")

In [24]:
time_data=os.path.join(output_data, "time")
time_table.write.partitionBy('year', 'month').parquet(time_data,mode="overwrite")

In [25]:
df.createOrReplaceTempView("logs")
time_table.createOrReplaceTempView("time")

In [26]:
songplays_table = spark.sql("""
    SELECT DISTINCT
        l.datetime as start_time,
        l.userId as user_id,
        l.level,
        s.song_id,
        s.artist_id,
        l.sessionId as session_id,
        l.location,
        l.userAgent as user_agent,
        t.year,
        t.month
    FROM logs l
    JOIN songs s on l.artist = s.artist_name and l.song = s.title and l.length = s.duration
    JOIN time t on t.start_time = l.datetime
""")

In [73]:
#songplays_table.write.partitionBy("year", "month").mode('overwrite').parquet("songplays.parquet")

In [27]:
songplays_data=os.path.join(output_data, "songplays")
songplays_table.write.partitionBy('year', 'month').parquet(songplays_data,mode="overwrite")